## Code to pull pixel brightness values along centerlines

In [14]:
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.pylab as pl
import matplotlib.image as mpimg
import cv2
import pandas as pd    
import PIL

csvpaths = '/home/jukes/Documents/Sample_glaciers/'
basepath = '/media/jukes/jukes1/LS8aws/'
massorsize = "mass"

Grab centerline info and BoxIDs

In [15]:
centerline_df = pd.read_csv(csvpaths+'Boxes_coords_pathrows_SE.csv', sep=',', dtype=str, header=0)
centerline_df = centerline_df.set_index('BoxID')
print(centerline_df.shape)

BoxIDs = list(centerline_df.index)
print(BoxIDs)

centerline_df

(14, 4)
['147', '148', '149', '150', '152', '190', '191', '192', '193', '194', '195', '196', '213', '214']


,midpoint_x,midpoint_y,cline_slope,cline_intercept
BoxID,,,,
147,63.0,91.5,0.49230769230769234,60.48461538461538
148,68.5,94.0,-0.2535211267605634,111.36619718309859
149,40.5,71.5,-0.17045454545454544,78.4034090909091
150,40.0,68.5,0.08791208791208792,64.98351648351648
152,63.5,87.0,0.3445378151260504,65.1218487394958
190,142.5,219.5,-0.18461538461538463,245.80769230769232
191,116.5,180.5,-0.021929824561403508,183.0548245614035
192,47.5,58.5,0.5131578947368421,34.125
193,71.0,155.0,-0.2623762376237624,173.62871287128712


In [19]:
#make lists to store image data and grab image files
ratios = []
imagenames = []

for BoxID in BoxIDs:
#     if BoxID == '001':
        imagepath = basepath+"Box"+BoxID+"/resized/"
        #image path
        imgfiles = os.listdir(imagepath)

        #grab terminus box raster
#         tbox = mpimg.imread(csvpaths+"Box"+BoxID+"/rotated_images/final/crop_Box"+BoxID+"_raster_cut.png")
#         tbox = mpimg.imread(imagepath+"crop_R_Box"+BoxID+"_cut.png")

        #grab centerline points
        xmin = float(centerline_df.loc[BoxID]['midpoint_x'])
        xmax = np.max(np.array(list(set(np.nonzero(tbox)[0]))))
        c_x = np.linspace(xmin, xmax, int(xmax-xmin))
        c_y = float(centerline_df.loc[BoxID]['cline_slope'])*c_x + float(centerline_df.loc[BoxID]['cline_intercept'])

        #load in each image and grab the pixel values indexed by the centerline points:
        for img in imgfiles:
            if img.endswith(BoxID+'.pgm'):

                imagename_pgm = img[:-4]+'.pgm'
                #load in the image using cv2
                image = cv2.imread(imagepath+img,-1)
                imageplt = mpimg.imread(imagepath+img)

                pixelvalues = []
                indexes = []

                #loop through all centerline coordinates
                for i in range(0, len(c_x)):
                    x = int(c_x[i])
                    y = int(c_y[i])
                    pixval = '%.3f' % image[y][x]
                    pixelvalues.append(float(pixval))
                    indexes.append(i)#*15.0)
                    
#                     print(x, y, pixval)

#                 #grab the first and last third
                third = int(len(pixelvalues)/3)
                third1 = pixelvalues[:third]
                third3 = pixelvalues[-third:]
#                 print(third1)
#                 print(third3)
                
#                 fig, ax1 = plt.subplots(figsize=(8,4))
#                 plt.plot(seaice_idxs, seaice_pixvals, 'b--', alpha=0.5)
#                 plt.plot(indexes, pixelvalues, 'k-')
# #                 ax1.set_title(img, fontsize=16)
#                 ax1.set_ylabel('Pixel Value', fontsize=12)
#                 ax1.set_xlabel('Distance along centerline (pixels)', fontsize=12)
#                 ax1.tick_params(axis='both', which='major', labelsize=12)
#                 plt.legend(['sea ice', 'open water'])
#                 plt.savefig(csvpaths+'Figures/Centerline_values_example_Box001.png', dpi=500)
#                 plt.show()

                #grab the average of the first and average of the last to compare
                avgthird1 = float(np.average(np.array(third1)))
                avgthird3 = float(np.average(np.array(third3)))
                ratio = "%.2f" % (avgthird1/avgthird3)
                
                ratios.append(ratio)
                imagenames.append(imagename_pgm)
#                 print('Ratio 1st/3rd:', ratio)
#                 print(img)

#                 plt.figure(figsize=(20,10))
#                 imgplt_trim = plt.imshow(cv2.cvtColor(imageplt, cv2.COLOR_BGR2RGB))
#                 plt.plot(c_x, c_y, '--w', linewidth=1.5, alpha=0.3)
#                 plt.text(int(0.02*image.shape[0]), int(0.035*image.shape[1]), ratio
#                          , fontsize=16, color='w')
#                 plt.show()
        
pixels_df = pd.DataFrame(list(zip(imagenames, ratios)))
pixels_df

IndexError: index 191 is out of bounds for axis 0 with size 191

In [49]:
# seaice_pixvals = pixelvalues
# seaice_idxs = indexes

In [8]:
#write to txt file
pixels_df.to_csv(path_or_buf=csvpaths+'Pixelratios_SE.txt', sep=' ', index=False, header=False)